# 4. Citybenches: Implementation

In [1]:
#Necessary packages
import osmnx as ox
import networkx as nx
import geopandas 
import matplotlib.pyplot as plt
import pandas as pd

## Results of Data Wrangling Round 1

In [ ]:
#Import data frame, adding 'New York City' to every field in Address-column
df = pd.read_excel('citybenchlocationsv03.xlsx')
df['Address'] = df['Address'] + ', New York City'

In [ ]:
#Subset original data set
addressborough = df[['Address', 'Borough']]

In [ ]:
#Further subsetting
addressm = addressborough.loc[addressborough['Borough'] == 'Manhattan']

## 4.1 Data Wrangling: Round 2

First we have to add a column with geocodes to the data frame. We have chosen a try-and-except-loop so that the loop will not be interrupted in case there are addresses that cannot be geocoded. If an address cannot be geocoded, the field in the geocode-column is filled by a whitespace.  

In [ ]:
#Adding a column with geocodes to df
addressgeo = []
for i in addressm['Address']:
    try:
        addressgeo.append(ox.geocode(i))
    except:
        addressgeo.append(' ')
        pass
addressm.insert(2, 'Geocode', addressgeo, True)

Because the geocoding process needs a lot of time, we saved it as a .pkl so we need not to repeat this process every time we launch the jupyter notebook. 

In [2]:
addressm = pd.read_pickle("addressm.pkl")

In [ ]:
#data = addressm.to_pickle("addressm.pkl")

Now we check all the fields in the geocode-column that are filled with whitespaces. The addresses that belong to these fields are the addresses that cannot be geocoded by ox.geocode(), so we need to find their geocode manually using the google maps user interface. 

In [ ]:
#Checking Geocoder
fgeocode = []
for i in range(len(addressm)):
    if addressm.iloc[i]['Geocode'] == ' ':
        fgeocode.append(addressm.index[i])       
print(fgeocode)

In [ ]:
# Buggy addresses
for i in fgeocode:
    print(addressm.loc[i]['Address'])

Below, we have written down the result of geocoding the "buggy" addresses manually using the google maps user interface:

In [15]:
#Maually corrected Geocodes
addressm.iloc[49]['Geocode'] = (40.800113, -73.940607)
addressm.iloc[52]['Geocode'] = (40.731681, -73.981837)
addressm.iloc[67]['Geocode'] = (40.799129, -73.941801)
addressm.iloc[69]['Geocode'] = (40.795961, -73.934174)
addressm.iloc[70]['Geocode'] = (40.795961, -73.934174)
addressm.iloc[97]['Geocode'] = (40.744852, -73.976229)
addressm.iloc[101]['Geocode'] = (40.724333, -73.992682)
addressm.iloc[117]['Geocode'] = (40.754906, -73.996792)
addressm.iloc[118]['Geocode'] = (40.754906, -73.996792)
addressm.iloc[162]['Geocode'] = (40.765892, -73.954634)
addressm.iloc[163]['Geocode'] = (40.765892, -73.954634)
addressm.iloc[226]['Geocode'] = (40.780839, -73.949743)
addressm.iloc[231]['Geocode'] = (40.774695, -73.986728)
addressm.iloc[240]['Geocode'] = (40.813993, -73.956925)
addressm.iloc[242]['Geocode'] = (40.819488, -73.944606)
addressm.iloc[260]['Geocode'] = (40.724082, -74.006416)
addressm.iloc[261]['Geocode'] = (40.724082, -74.006416)
addressm.iloc[262]['Geocode'] = (40.724262, -74.007545)
addressm.iloc[263]['Geocode'] = (40.724262, -74.007545)
addressm.iloc[265]['Geocode'] = (40.714231, -73.994227)
addressm.iloc[266]['Geocode'] = (40.714231, -73.994227)
addressm.iloc[312]['Geocode'] = (40.823554, -73.937335)
addressm.iloc[315]['Geocode'] = (40.772293, -73.984000)
addressm.iloc[342]['Geocode'] = (40.829248, -73.936084)
addressm.iloc[356]['Geocode'] = (40.830368, -73.939648)
addressm.iloc[430]['Geocode'] = (40.707767, -73.990772)
addressm.iloc[431]['Geocode'] = (40.707767, -73.990772)
addressm.iloc[432]['Geocode'] = (40.707767, -73.990772)
addressm.iloc[433]['Geocode'] = (40.707767, -73.990772)
addressm.iloc[468]['Geocode'] = (40.772838, -73.983241)
addressm.iloc[484]['Geocode'] = (40.733484, -73.980689)
addressm.iloc[489]['Geocode'] = (40.742792, -73.996511)

In [3]:
#Choosing a graph for Manhattan
G = ox.graph_from_place('Manhattan, New York City', network_type = 'walk', simplify = False)

Now we have to add annother column for the nodes of the geocodes within the graph.

In [ ]:
#Adding a column with Nodes to df
addressnode = []
for i in addressm['Geocode']:
    addressnode.append(ox.get_nearest_node(G, i))  
addressm.insert(3, 'Node', addressnode, True)

In [ ]:
#Save data frame
addressm.to_pickle('addressm.pkl')

In [ ]:
addressm.head(29)

Finally, we have to calculate the shortest distance. Unfortunately, this task demands so much computing power that have not finished it yet. 

In [ ]:
#Calculating the shortest distance
distancel = []
counter1 = 0 
for i in addressm['Node']:
    counter1 += 1
    print(counter1)
    distance = []
    for j in addressm['Node']:
        distance.append(nx.shortest_path_length(G, i, j, weight = 'length'))
    distancel.append(distance)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [ ]:
#Saving the result
with open('distancel', 'wb') as f:
    pickle.dump(distancel, f)

tuple

In [4]:
import itertools

In [ ]:
a = [1,2,3]

In [5]:
Nodes = addressm['Node']

In [22]:
Nodesv01 = Nodes.head(10)

In [6]:
permutations_object = itertools.permutations(Nodes,2)

In [7]:
permutation_list = list(permutations_object)

In [8]:
len(permutation_list)

258572

In [8]:
#print(Nodesv01)

In [9]:
#print(permutation_list)

In [ ]:
distance = []
counter = 0
for i,j in permutation_list:
    try:
        d = nx.shortest_path_length(G, i, j, weight = 'length')
        counter += 1
        print(counter/258572)
        distance.append(d)
    except:
        distance.append(" ")
        pass

3.867394768188357e-06
7.734789536376715e-06
1.1602184304565072e-05
1.546957907275343e-05
1.933697384094179e-05
2.3204368609130144e-05
2.7071763377318503e-05
3.093915814550686e-05
3.480655291369522e-05
3.867394768188358e-05
4.254134245007193e-05
4.640873721826029e-05
5.027613198644865e-05
5.4143526754637006e-05
5.801092152282536e-05
6.187831629101372e-05
6.574571105920208e-05
6.961310582739044e-05
7.348050059557879e-05
7.734789536376715e-05
8.12152901319555e-05
8.508268490014387e-05
8.895007966833223e-05
9.281747443652058e-05
9.668486920470894e-05
0.0001005522639728973
0.00010441965874108565
0.00010828705350927401
0.00011215444827746238
0.00011602184304565072
0.00011988923781383909
0.00012375663258202744
0.0001276240273502158
0.00013149142211840416
0.00013535881688659252
0.00013922621165478089
0.00014309360642296922
0.00014696100119115758
0.00015082839595934595
0.0001546957907275343
0.00015856318549572267
0.000162430580263911
0.00016629797503209937
0.00017016536980028773
0.0001740327645

In [ ]:
#Saving the result
import pickle
with open('distance.txt', 'wb') as fp:
    pickle.dump(distance, fp)

In [29]:
with open("distance.txt", "rb") as fp:
    b = pickle.load(fp)


In [30]:
print(b)

[9994.472000000002, 9925.326000000001, 9773.585000000001, 9773.585000000001, 8551.925999999994, 5455.073999999999, 3323.094000000002, 2902.405000000001, 4030.3600000000024, 9994.472000000012, 89.28800000000001, 246.066, 246.066, 1703.5310000000006, 5582.275999999998, 6817.240000000003, 8758.687000000007, 6048.001000000005, 9925.326000000012, 89.288, 156.778, 156.778, 1614.2430000000006, 5492.987999999998, 6748.094000000003, 8669.399000000007, 5978.855000000004, 9773.585000000012, 246.066, 156.778, 0, 1457.4650000000006, 5336.209999999997, 6596.353000000003, 8512.621000000005, 5827.114000000004, 9773.585000000012, 246.066, 156.778, 0, 1457.4650000000006, 5336.209999999997, 6596.353000000003, 8512.621000000005, 5827.114000000004, 8551.926000000005, 1703.531, 1614.243, 1457.4650000000001, 1457.4650000000001, 3887.6059999999998, 5402.029999999999, 7055.1560000000045, 4632.791, 5455.074, 5582.276000000001, 5492.988000000001, 5336.210000000001, 5336.210000000001, 3887.605999999999, 2661.568,

In [53]:
def dist_calc(i,j):
    c = nx.shortest_path_length(G, i, j, weight = 'length')
    return c

In [54]:
import multiprocessing as mp

In [57]:
pool = mp.Pool(mp.cpu_count())

In [56]:
results = [pool.apply(dist_calc, args=(i,j)) for i,j in permutation_list]

pool.close()
print(results)

[9994.472000000002, 9925.326000000001, 9773.585000000001, 9773.585000000001, 8551.925999999994, 5455.073999999999, 3323.094000000002, 2902.405000000001, 4030.3600000000024, 9994.472000000012, 89.28800000000001, 246.066, 246.066, 1703.5310000000006, 5582.275999999998, 6817.240000000003, 8758.687000000007, 6048.001000000005, 9925.326000000012, 89.288, 156.778, 156.778, 1614.2430000000006, 5492.987999999998, 6748.094000000003, 8669.399000000007, 5978.855000000004, 9773.585000000012, 246.066, 156.778, 0, 1457.4650000000006, 5336.209999999997, 6596.353000000003, 8512.621000000005, 5827.114000000004, 9773.585000000012, 246.066, 156.778, 0, 1457.4650000000006, 5336.209999999997, 6596.353000000003, 8512.621000000005, 5827.114000000004, 8551.926000000005, 1703.531, 1614.243, 1457.4650000000001, 1457.4650000000001, 3887.6059999999998, 5402.029999999999, 7055.1560000000045, 4632.791, 5455.074, 5582.276000000001, 5492.988000000001, 5336.210000000001, 5336.210000000001, 3887.605999999999, 2661.568,

In [59]:
results = pool.map(dist_calc,[for i,j in permutation_list])

pool.close()
print(results)

SyntaxError: invalid syntax (<ipython-input-59-5c11b7b13097>, line 1)

In [ ]:
# Parallel processing with Pool.apply_async()

import multiprocessing as mp
pool = mp.Pool(mp.cpu_count())

results = []

# Step 1: Redefine, to accept `i`, the iteration number
def howmany_within_range2(i, row, minimum, maximum):
    """Returns how many numbers lie within `maximum` and `minimum` in a given `row`"""
    count = 0
    for n in row:
        if minimum <= n <= maximum:
            count = count + 1
    return (i, count)


# Step 2: Define callback function to collect the output in `results`
def collect_result(result):
    global results
    results.append(result)


# Step 3: Use loop to parallelize
for i, row in enumerate(data):
    pool.apply_async(howmany_within_range2, args=(i, row, 4, 8), callback=collect_result)

# Step 4: Close Pool and let all the processes complete    
pool.close()
pool.join()  # postpones the execution of next line of code until all processes in the queue are done.

# Step 5: Sort results [OPTIONAL]
results.sort(key=lambda x: x[0])
results_final = [r for i, r in results]

print(results_final[:10])